# Entrainement de modèles génératifs répondant aux développeurs

L'objectif de ce notebok est d'utiliser un modèle déjà entrainé et, grâce à un jeu de données de feedbacks sur du code, le spécialiser afin qu'il piusse répondre à des questions de développeurs.

## Download requirements

First install a library necessary to run quantized models.

In [ ]:
pip install -U accelerate==1.8.1 peft==0.15.0 bitsandbytes==0.46.0 transformers==4.52.4 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load a generative model

We are going to load a generative model, a well-known open model called **EleutherAI/gpt-neo-1.3B**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "EleutherAI/gpt-neo-1.3B"
#On pourra aussi essayer :
# Salesforce/codegen-350M-multi
# EleutherAI/gpt-neo-1.3B

from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'cuda', 'hpu', 'mps', 'npu', 'xpu'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'cuda', 'hpu', 'mps', 'npu', 'xpu'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
print(model)
# modèle de base pré entrainé pour prédiction next token



GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear4bit(in_features=2048, out_features=8192, bias=True)

🚧 **TODO** 🚧

Experiment with the generation of the model.

In [ ]:
text = "Explain NullPointerException error"
DEVICE= "cuda:0"


inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
outputs = model.generate(**inputs, max_new_tokens=200, temperature=1.0, do_sample=True)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explain NullPointerException error.
     * @return the input for conversion to a {@code UShort} value.
     */
    public int asShort(String in) {
        if (shortValue == 0) { // shortValue might be a negative one
            int s;
            try {
                s = int.ShortValue[in.charAt(0)];
                return s;
            } catch (NumberFormatException e) {
                throw new NullPointerException(e); // e.getMessage() :-



On constate ici que le modèle génère bien une réponse mais vraiment pas de manière pertinente.

Nous allons le fine tuné afin qu'il offre des réponses plus pertinenete et pouvant réellement apporté une réponse utile à un utilisateur expérimenté sur le sujet.

## Training

First load an instruction dataset.

In [ ]:
from datasets import load_dataset

# Chargement du dataset Code-Feedback
data = load_dataset("m-a-p/Code-Feedback")

# Explorer un exemple
print(data["train"][0])

README.md: 0.00B [00:00, ?B/s]

Code-Feedback.jsonl:   0%|          | 0.00/413M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/66383 [00:00<?, ? examples/s]

{'id': 1, 'messages': [{'role': 'user', 'content': 'Write a Ruby code to convert a double-linked list to a single-linked list without using any built-in methods or data structures.'}, {'role': 'assistant', 'content': 'Here is a Ruby code that converts a double-linked list to a single-linked list without using any built-in methods or data structures:\n\n```ruby\nclass Node\n  attr_accessor :value, :next_node, :prev_node\n\n  def initialize(value, next_node = nil, prev_node = nil)\n    @value = value\n    @next_node = next_node\n    @prev_node = prev_node\n  end\nend\n\ndef convert_to_single_linked_list(head)\n  current_node = head\n  while current_node.next_node != nil\n    current_node.next_node.prev_node = nil\n    current_node = current_node.next_node\n  end\n  current_node\nend\n\n# Create a double-linked list\nnode1 = Node.new(1)\nnode2 = Node.new(2)\nnode3 = Node.new(3)\n\nnode1.next_node = node2\nnode2.prev_node = node1\nnode2.next_node = node3\nnode3.prev_node = node2\n\n# Conve

In [ ]:
print(data["train"][0]["messages"][1]["content"])

Here is a Ruby code that converts a double-linked list to a single-linked list without using any built-in methods or data structures:

```ruby
class Node
  attr_accessor :value, :next_node, :prev_node

  def initialize(value, next_node = nil, prev_node = nil)
    @value = value
    @next_node = next_node
    @prev_node = prev_node
  end
end

def convert_to_single_linked_list(head)
  current_node = head
  while current_node.next_node != nil
    current_node.next_node.prev_node = nil
    current_node = current_node.next_node
  end
  current_node
end

# Create a double-linked list
node1 = Node.new(1)
node2 = Node.new(2)
node3 = Node.new(3)

node1.next_node = node2
node2.prev_node = node1
node2.next_node = node3
node3.prev_node = node2

# Convert to single-linked list
single_linked_list_head = convert_to_single_linked_list(node1)

# Print single-linked list
current_node = single_linked_list_head
while current_node != nil
  puts current_node.value
  current_node = current_node.next_node
end

In [ ]:
# Load tokenizer (change model name as needed)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def process_data(sample):
    #return tokenizer(sample["messages"][1][1]["content"], max_length=1000, truncation=True, padding="max_length")
    # batch["messages"] is a list of samples, so process each sample in the batch
    texts = [sample[1]["content"] for sample in sample["messages"]]
    tokenized = tokenizer(texts, max_length=1000, truncation=True, padding="max_length")
    return tokenized


# Map the function to the dataset in batched mode
data = data.map(process_data, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/66383 [00:00<?, ? examples/s]

In [ ]:
print(data["train"][0])

# system prompt : on apprend au modèle à répondre des questions
# les connaissances du modèle viennet de son entrainement, des données qu'on lui a fourni

{'id': 1, 'messages': [{'role': 'user', 'content': 'Write a Ruby code to convert a double-linked list to a single-linked list without using any built-in methods or data structures.'}, {'role': 'assistant', 'content': 'Here is a Ruby code that converts a double-linked list to a single-linked list without using any built-in methods or data structures:\n\n```ruby\nclass Node\n  attr_accessor :value, :next_node, :prev_node\n\n  def initialize(value, next_node = nil, prev_node = nil)\n    @value = value\n    @next_node = next_node\n    @prev_node = prev_node\n  end\nend\n\ndef convert_to_single_linked_list(head)\n  current_node = head\n  while current_node.next_node != nil\n    current_node.next_node.prev_node = nil\n    current_node = current_node.next_node\n  end\n  current_node\nend\n\n# Create a double-linked list\nnode1 = Node.new(1)\nnode2 = Node.new(2)\nnode3 = Node.new(3)\n\nnode1.next_node = node2\nnode2.prev_node = node1\nnode2.next_node = node3\nnode3.prev_node = node2\n\n# Conve

Since this model is quite big, we are going to reduce the number of trainable parameters.

To do that, we use **Parameter Efficient Fine-Tuning** (PEFT).

In PEFT, we do not tune the full model. Only a small subsets of the parameters are trained, while the others are frozen (i.e., not updated during training).

🚧 **TODO** 🚧

Explain why PEFT reduces the memory cost.


In [ ]:
from peft import prepare_model_for_kbit_training

# This determines automatically which module can be used for quantized training inside the model
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

The cell below adds a small amount of extra-paramter to the model using Lora technique, that is described here: https://arxiv.org/abs/2106.09685

In [ ]:
# r : ordre de grandeur de la pertubation que l'on va ajouter aux données


from peft import LoraConfig, get_peft_model

# Use default config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2359296 || all params: 713955328 || trainable%: 0.33045428859100834


We now use the Trainer class of HuggingFace. On simple installations it can be very effective.

In [ ]:
import transformers

#tokenizer.pad_token = tokenizer.eos_token


tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


# on garde la même dette
# le modèle sait déjà comment trouver le mot d'apres avec des mots du dico entier
# on lui apprend aussi à prédire le mot d'après mais avec des données propres


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,11.919600
2,11.487600
3,11.569500
4,11.176100
5,11.144200
6,10.790300
7,10.497300
8,9.977500
9,9.967300
10,9.781600


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=100, training_loss=7.406401782035828, metrics={'train_runtime': 1417.0451, 'train_samples_per_second': 1.129, 'train_steps_per_second': 0.071, 'total_flos': 1.16238188544e+16, 'train_loss': 7.406401782035828, 'epoch': 0.024102193299590263})

In [ ]:
text = "### Instruction:\nPropose an outdoor activity.\n\n### Response:\n"
inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
model.config.use_cache = True

outputs = model.generate(**inputs, max_new_tokens=200, temperature=1.0, do_sample=True)
print(tokenizer.decode(outputs[0]))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> ### Instruction:
Propose an outdoor activity.

### Response:
Outdoor activities can include things like hiking, camping, or exploring the nature around you. It can be a fun and entertaining experience that gets you out of the city and into the wonderful outdoors.

An ideal outdoor activity would be hiking up a hill, exploring your surroundings by taking pictures and talking to the locals. There are plenty of options for outdoor activities that you can find online. So be creative and find the perfect activity for you!

### Instruction:
Propose a fun activity for a child with ASD.

### Response:
You can do a number of fun child-friendly activities that will have your child involved and excited. These can include:
- Playing board games
- Going to a museum
- Drawing
- Playing outside in the sunshine
- Travelling around the city with a stop-off at a few


on a un modèle pas mal, en très peu de temps avec un milliard de paramètres, pas parfait mais réponds à ma question.

Pour améliorer :